## First Part

In [1]:
# Transportation Problem: data
Plants = ['Plant 1', 'Plant 2']
Customers = ['Cust A', 'Cust B', 'Cust C']

Cost = {
    ('Plant 1', 'Cust A') : 15,
    ('Plant 1', 'Cust B') : 35,
    ('Plant 1', 'Cust C') : 25,
    ('Plant 2', 'Cust A') : 10,
    ('Plant 2', 'Cust B') : 30,
    ('Plant 2', 'Cust C') : 40
}

Supply = {
    'Plant 1' : 40,
    'Plant 2' : 50    
}

Demand = {
    'Cust A' : 30,
    'Cust B' : 30,
    'Cust C' : 30,
}

#### Implicit Representation of the Model

In [2]:
from docplex.mp.model import Model
from docplex.util.environment import get_environment

In [3]:
mdl = Model()

In [4]:
amount_shipped = mdl.continuous_var_matrix(Plants, Customers, lb = 0, name='amount_shipped')

In [5]:
mdl.minimize(mdl.sum(amount_shipped[(p,c)] * Cost[(p,c)] for p in Plants for c in Customers))

In [6]:
for c in Customers:
    mdl.add_constraint(mdl.sum(amount_shipped[(p,c)] for p in Plants) == Demand[c])

In [7]:
for p in Plants:
    mdl.add_constraint(mdl.sum(amount_shipped[(p,c)] for c in Customers) == Supply[p])

In [8]:
mdl.solve()

docplex.mp.solution.SolveSolution(obj=2000,values={amount_shipped_Plant ..

In [9]:
mdl.get_solve_details()

docplex.mp.SolveDetails(time=0,status='optimal')

In [10]:
mdl.print_solution()

objective: 2000.000
status: OPTIMAL_SOLUTION(2)
  "amount_shipped_Plant 1_Cust B"=10.000
  "amount_shipped_Plant 1_Cust C"=30.000
  "amount_shipped_Plant 2_Cust A"=30.000
  "amount_shipped_Plant 2_Cust B"=20.000


## Second Part (Variant)

In [11]:
# Transportation Problem: data

# For this variant we introduce a 'dummy' plant to represent unmet demand (10 units)
Plants = ['Plant 1', 'Plant 2', 'Plant 3']
Customers = ['Cust A', 'Cust B', 'Cust C']

Cost = {
    ('Plant 1', 'Cust A') : 15,
    ('Plant 1', 'Cust B') : 35,
    ('Plant 1', 'Cust C') : 25,
    ('Plant 2', 'Cust A') : 10,
    ('Plant 2', 'Cust B') : 30,
    ('Plant 2', 'Cust C') : 40,
    ('Plant 3', 'Cust A') : 20,
    ('Plant 3', 'Cust B') : 10,
    ('Plant 3', 'Cust C') : 30
}

Supply = {
    'Plant 1' : 30,
    'Plant 2' : 50,
    'Plant 3' : 10
}

Demand = {
    'Cust A' : 30,
    'Cust B' : 30,
    'Cust C' : 30,
}

In [12]:
mdl = Model()

In [13]:
# for this example we use a 2-dimensional matrix of variables: 
shipped = mdl.continuous_var_matrix(Plants, Customers, lb=0, name='shipped')

In [14]:
# the objective is to minimize the total weighted shipping costs:
objective = mdl.sum(shipped[(p,c)]*Cost[(p,c)] for p in Plants for c in Customers)
mdl.minimize(objective)

In [15]:
# define the demand constraints:
for c in Customers:
    mdl.add_constraint(mdl.sum(shipped[(p,c)] for p in Plants) == Demand[c])

In [16]:
# define the supply constraints:
for p in Plants:
    mdl.add_constraint(mdl.sum(shipped[(p,c)] for c in Customers) == Supply[p])

In [17]:
# Further restrictions: ship at least 5 units from 1 to B; ship at most 20 units from 1 to C:
mdl.add_constraint(shipped[('Plant 1', 'Cust B')] >= 5)
mdl.add_constraint(shipped[('Plant 1', 'Cust C')] <= 20)

docplex.mp.LinearConstraint[](shipped_Plant 1_Cust C,LE,20)

In [18]:
mdl.solve()
mdl.get_solve_details()

docplex.mp.SolveDetails(time=0,status='optimal')

In [19]:
mdl.print_solution()

objective: 1950.000
status: OPTIMAL_SOLUTION(2)
  "shipped_Plant 1_Cust A"=5.000
  "shipped_Plant 1_Cust B"=5.000
  "shipped_Plant 1_Cust C"=20.000
  "shipped_Plant 2_Cust A"=25.000
  "shipped_Plant 2_Cust B"=15.000
  "shipped_Plant 2_Cust C"=10.000
  "shipped_Plant 3_Cust B"=10.000
